# Project: IT Support Analyst with LangChain and Prompt Engineering
* Notebook by Adam Lang
* Date: 6/27/2024

# Overview
* This is a mini project using GPT models and LangChain to build an IT support analyst.
* We will use a JSON output parser with Pydantic to structure the specific data structure output.

## Install dependencies

In [1]:
# imports
!pip install langchain==0.1.19
!pip install langchain-openai==0.1.6
!pip install langchain-community==0.0.38

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.9/302.9 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.4/127.4 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.0/145.0 kB 2.3 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 24.1
    Uninstalling packaging-24.1:
      Successfully uninstalled packaging-24.1
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 327.5/327.5 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

## API Tokens
* OpenAI access

In [2]:
# enter api key
from getpass import getpass


OPENAI_KEY = getpass('Enter your Open AI key: ')

Enter your Open AI key: ··········


In [3]:
## openai environment variables
import os



os.environ['OPENAI_API_KEY'] = OPENAI_KEY

## LLM Dependencies

In [4]:
## imports
from langchain_core.prompts import ChatPromptTemplate # prompt templates from langchain
from langchain_openai import ChatOpenAI

In [5]:
## chatgpt instance
chatgpt = ChatOpenAI(model_name='gpt-3.5-turbo', temperature=0)

## Project: IT Support Analyst
Ask ChatGPT to act as a IT support agent, process each customer IT ticket message and output the response in JSON with the following fields

```
orig_msg: The original customer message
orig_lang: Detected language of the customer message e.g. Spanish
category: 1-2 word describing the category of the problem
trans_msg: Translated customer message in English
response: Response to the customer in orig_lang
trans_response: Response to the customer in English
```

Try to use a JSON parser to get the responses in JSON for each ticket

## Define Output Parser
* JSON output parser
* Pydantic

In [6]:
## imports
from langchain_core.prompts import PromptTemplate #prompt template
from langchain_core.output_parsers import JsonOutputParser #JSON output parser
from langchain_core.pydantic_v1 import BaseModel, Field

# Define Pydantic class with core data structures for serialization
class ITSupportResponse(BaseModel):
  orig_msg: str = Field(description="The original customer IT support query message.")
  orig_lang: str = Field(description="Detected language of the customer message, e.g. Spanish, French")
  category: str = Field(description="1 to 2 words describing the category of the problem.")
  trans_msg: str = Field(description="Translated customer IT support query message in English.")
  response: str = Field(description="Response to the customer in their original language - orig_lang")
  trans_response: str = Field(description="Response to the customer in English.")


# output parser
parser = JsonOutputParser(pydantic_object=ITSupportResponse)

## Create LLM input prompt

In [7]:
## final prompt formatting instructions from parser
prompt_txt = """
             You are an experienced Information Technology (IT) customer support agent.
             For the IT suppor message mentioned below,
             use the following output format when generating the output response:

             Output format instructions:
             {format_instructions}

             Customer IT support message:
             {it_support_msg}
            """

## prompt template
prompt = PromptTemplate(
    template=prompt_txt,
    input_variables=['it_support_msg'],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

## LCEL LLM Chain

In [8]:
## LCEL chain with prompt --> to LLM --> output parser
llm_chain = (prompt
               |
             chatgpt
               |
             parser)

## Data for Customer IT Support Ticket

In [9]:
it_support_queue = [
    "Não consigo sincronizar meus contatos com o telefone. Sempre recebo uma mensagem de falha.",
    "Ho problemi a stampare i documenti da remoto. Il lavoro non viene inviato alla stampante di rete.",
    "プリンターのトナーを交換しましたが、印刷品質が低下しています。サポートが必要です。",
    "Я не могу войти в систему учета времени, появляется сообщение об ошибке. Мне нужна помощь.",
    "Internet bağlantım çok yavaş ve bazen tamamen kesiliyor. Yardım eder misiniz?",
    "Не могу установить обновление безопасности. Появляется код ошибки. Помогите, пожалуйста."
]

## format message
formatted_msgs = [{"it_support_msg": msg}
                    for msg in it_support_queue]
formatted_msgs[0]

{'it_support_msg': 'Não consigo sincronizar meus contatos com o telefone. Sempre recebo uma mensagem de falha.'}

## LLM responses

In [10]:
## responses
responses = llm_chain.map().invoke(formatted_msgs)

## Get responses from LLM

In [11]:
responses[0]

{'orig_msg': 'Não consigo sincronizar meus contatos com o telefone. Sempre recebo uma mensagem de falha.',
 'orig_lang': 'Portuguese',
 'category': 'Sync Issue',
 'trans_msg': "I can't sync my contacts with the phone. I always receive a failure message.",
 'response': 'Por favor, verifique se a conexão Bluetooth está ativada no seu telefone e se os contatos estão configurados para sincronização. Se o problema persistir, entre em contato com o suporte técnico para obter assistência adicional.',
 'trans_response': 'Please check if Bluetooth connection is enabled on your phone and if contacts are set up for synchronization. If the issue persists, please contact technical support for further assistance.'}

In [13]:
## type of response
type(responses[0])

dict

In [14]:
## transpose to df
import pandas as pd

df = pd.DataFrame(responses)
df

,orig_msg,orig_lang,category,trans_msg,response,trans_response
0,Não consigo sincronizar meus contatos com o te...,Portuguese,Sync Issue,I can't sync my contacts with the phone. I alw...,"Por favor, verifique se a conexão Bluetooth es...",Please check if Bluetooth connection is enable...
1,Ho problemi a stampare i documenti da remoto. ...,Italian,Printing,I am having trouble printing documents remotel...,Grazie per averci contattato. Per risolvere il...,Thank you for contacting us. To resolve the is...
2,プリンターのトナーを交換しましたが、印刷品質が低下しています。サポートが必要です。,Japanese,Printer,"I replaced the toner of the printer, but the p...",トナーを正しく交換したことを確認してください。プリンターのメンテナンスを行うことをお勧めしま...,Please make sure you replaced the toner correc...
3,"Я не могу войти в систему учета времени, появл...",Russian,Login Issue,"I can't log into the time tracking system, an ...","Пожалуйста, попробуйте сбросить пароль и попро...",Please try resetting your password and attempt...
4,Internet bağlantım çok yavaş ve bazen tamamen ...,Turkish,Internet Connectivity,My internet connection is very slow and someti...,"Evet, tabii ki yardımcı olabiliriz. İnternet b...","Yes, of course we can help. We will assist you..."
5,Не могу установить обновление безопасности. По...,Russian,Installation,Unable to install security update. Error code ...,"Извините за возникшие проблемы. Пожалуйста, по...",Apologies for the inconvenience. Please try th...
